In [2]:
# IMPORT LIBRARIES #
import pandas as pd
import pygsheets
import numpy as np

In [3]:
# FLAGS #

#Set to 1 to print diagnostic info
debugFlag = 0
#Set to 1 to pull info from RawNames sheet, join GPS, update, and export
updateGPSFlag = 0

In [4]:
# IMPORT DATA #

gc = pygsheets.authorize(service_file='file.json')
sh = gc.open('DashData')

if debugFlag:
    for x in sh:
        print(x)

for x in sh:
    title = x.title
    if(title=="DeliveriesOld"):
        import_sheets_OldDelDF = x.get_as_df()
    elif(title=="Deliveries"):
        import_sheets_DelDF = x.get_as_df()
    elif(title=="Days"):
        import_sheets_DaysDF = x.get_as_df()
    elif(title=="Weeks"):
        import_sheets_WeeksDF = x.get_as_df()
    elif(title=="RawNames"):
        import_sheets_RawNamesDF = x.get_as_df()
    elif(title=="GPS"):
        #for import
        import_sheets_GPSDF = x.get_as_df()
        #for export
        import_wks_gps = x
    else:
        print("Unused worksheet detected: ", x)

Unused worksheet detected:  <Worksheet 'Import' index:6>
Unused worksheet detected:  <Worksheet 'Lifetime Deliveries' index:7>
Unused worksheet detected:  <Worksheet 'Days Old' index:8>
Unused worksheet detected:  <Worksheet 'Weeks Old' index:9>


In [5]:
# Allows for disjoint cells, one for import, one for reset to import value (for ease of testing)

#DF from sheets
sheets_OldDelDF   = import_sheets_OldDelDF.copy()
sheets_DelDF      = import_sheets_DelDF.copy()
sheets_DaysDF     = import_sheets_DaysDF.copy()
sheets_WeeksDF    = import_sheets_WeeksDF.copy()
sheets_RawNamesDF = import_sheets_RawNamesDF.copy()
sheets_GPSDF      = import_sheets_GPSDF.copy()

#Sheets, ah right can't use copy on a worksheet object, might need an alternate
#wks_gps           = import_wks_gps.copy()

In [6]:
# FUNCTIONS #

# TODO: develop function to take avg distance and duration for stacked orders

def dollars_to_floats(df, columns):
    for x in columns:
        df[x] = df[x].str.replace('$', '')
        df[x] = df[x].str.replace(',', '')
        df[x] = df[x].astype(float)
        
def hours_to_floats(df, columns):
    for x in columns:
        for index, row in df.iterrows():
            active = row[x]
            activeH = float(active[:active.index(":")])
            activeM = round(float(active[active.index(":")+1:active.index(":")+3])/60, 2)
            active = activeH + activeM
            df.at[index, x] = active
        df[x] = df[x].astype(float)
            
            

#Currently only valid for 2023
def date_to_timestamp(df, column):
    for index, row in df.iterrows():
        df.at[index, column] = pd.Timestamp('2023/' + row[column])
        
        
#Currently only valid for 2023
# Currently unused function
# Would we actually want this to have different dates? it's really the date the shift started
# So maybe, keep date same, but certain analysis might require that difference to be handled
def time_to_timestamp(df, DateCol, TimeCols):
    for index, row in df.iterrows():
        df.at[index, TimeCols[0]] = pd.Timestamp('2023/' + row[DateCol] + " " + row[TimeCols[0]])
        df.at[index, TimeCols[1]] = pd.Timestamp('2023/' + row[DateCol] + " " + row[TimeCols[1]])

In [7]:
# UPDATE GPS WORKSHEET #

if updateGPSFlag:
    testingDF = sheets_RawNamesDF.copy()
    testingDF = testingDF.drop_duplicates(subset=["Restaurant Name", "RAW DATA"])
    testingDF = testingDF[["Restaurant Name", "RAW DATA", "Note"]]
    testingDF = testingDF.sort_values(by=['Restaurant Name', "RAW DATA"])
    newNamesDF = testingDF.merge(sheets_GPSDF, how='left')
    for index, row in newNamesDF.iterrows():
        if isinstance(row['Address'], float):
            if not row['Note'] == "":
                newNamesDF.at[index, "Address"] = row["Note"]
            else:
                print("Note not found for:", index)
        if (row['Address'] == ""):
            newNamesDF.at[index, "Resolved"] = "0"
        else:
            newNamesDF.at[index, "Resolved"] = "1"
    newNamesDF = newNamesDF.drop(columns=["Note"])
    wks_gps.set_dataframe(newNamesDF,(1,1))

In [8]:
# COMBINE DELIVERY DATA #
sheets_frames = [sheets_OldDelDF, sheets_DelDF]
sheets_result = pd.concat(sheets_frames)
sheets_result.reset_index(drop=True, inplace=True)

In [9]:
# FUNCTIONS TO CONVERT DOLLAR AND HOUR VALUES TO FLOATS #


#All
dollars_to_floats(sheets_result, ["Total", "Base", "Tip", "Peak Bonus"])
#V3 only
dollars_to_floats(sheets_DelDF, ["Total", "Base", "Tip", "Peak Bonus"])
#Days
dollars_to_floats(sheets_DaysDF, ["Total", "Base", "Tip"])
hours_to_floats(sheets_DaysDF, ["Dash", "Active"])
#Weeks
dollars_to_floats(sheets_WeeksDF, ["Total", "Base", "Tip", "Adj pay", "Other", "Pre Total"])
hours_to_floats(sheets_WeeksDF, ["Dash", "Active"])

/tmp/ipykernel_4404/4229877212.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')


In [10]:
# BEGIN ADJ PAY SECTION #

# This establishes the estimated pay adjustment estimates by delivery, date, and week
# This is calculated using time spent delivering and distance approximations
# The real adjusted pay is based on weekly values
# I suspect the primary cause for difference between estimate and real values stems from 
#how I estimate mileage vs how the app tracks mileage

# DFs are copied to avoid excluding dropped values in later analysis



#if debugFlag:
#    sheets_DelDF.tail()

In [11]:
#Per delivery estimations
sheets_DelDF1 = sheets_DelDF.copy()

#Convert to float if possible, else convert to NaN
sheets_DelDF1["Duration"] = pd.to_numeric(sheets_DelDF1["Duration"], errors='coerce')
#Drop NaN rows
sheets_DelDF1 = sheets_DelDF1[sheets_DelDF1["Duration"].notna()]
#Calculate estimate adjustment per delivery
sheets_DelDF1["Est Adj"] = (sheets_DelDF1["Duration"]/60*18
                           + sheets_DelDF1["Distance"]*(0.34) ) - sheets_DelDF1["Base"]
#Duration is over 60 because its value is in minutes, values from day and week are in hours

#if debugFlag:
#    sheets_DelDF.tail()

In [12]:
#if debugFlag:
#    sheets_DaysDF.tail()

In [13]:
#Per day estimations
sheets_DaysDF1 = sheets_DaysDF.copy()
sheets_DaysDF1 = sheets_DaysDF1[sheets_DaysDF1["Version"] == 3]
sheets_DaysDF1["Est Adj"] = (sheets_DaysDF1["Active"]*(18) 
                            + sheets_DaysDF1["Active Mileage"]*(0.34) ) - sheets_DaysDF1["Base"]

In [14]:
#if debugFlag:
#    sheets_DaysDFv3.tail()

In [15]:
#Per week estimations
sheets_WeeksDF1 = sheets_WeeksDF.copy()
sheets_WeeksDF1 = sheets_WeeksDF1[sheets_WeeksDF1["Version"] == 3] #totals doesn't have v3 so its dropped
sheets_WeeksDF1["Est Adj"] = (sheets_WeeksDF1["Active"]*(18) 
                            + sheets_WeeksDF1["Active Mileage"]*(0.34) ) - sheets_WeeksDF1["Base"]

In [16]:
#if debugFlag:
#    sheets_WeeksDFv3.tail()

# Sample values

print("Del DF Sample\n")
print(sheets_DelDF1.iloc[-1])
print("\nDays DF Sample\n")
print(sheets_DaysDF1.iloc[-1])
print("\nWeeks DF Sample\n")
print(sheets_WeeksDF1.iloc[-1])
    
    
# END ADJ PAY SECTION #

Del DF Sample

ID                       1070
Date                      8/6
Restaurant Name           KFC
Total                    9.25
Base                     5.75
Tip                       3.5
Peak Bonus                0.0
Stacked                      
Start Time         6:43:00 PM
Distance                  4.9
End Time           7:04:00 PM
Duration                 21.0
Version                     3
Est Adj                 2.216
Name: 495, dtype: object

Days DF Sample

Date                           8/6
Total                         95.0
Base                          56.5
Tip                           38.5
Start (24)                   16:37
End (24)                     20:50
Active                        3.32
Dash                          4.22
Offers                          11
Deliveries                      11
Mileage Start                71454
Mileage End                  71510
Total Mileage                   56
Active Mileage                50.7
Version                          

In [17]:
#Percentage of deliveries taken that tip
numUntipped = (sheets_result["Tip"] == 0.0).sum()
numTipped = (sheets_result["Tip"] != 0.0).sum()
percTipped = round((numTipped/(numUntipped+numTipped)), 4)
print(percTipped*100, "% of customers tip")

90.74 % of customers tip


In [18]:
# Restaurants rankable by avg or mean base/tip/total
# Yes, total is almost always base+tip, but total INCLUDES peak pay, which may be desirable information

tempDF = sheets_result.copy()
#The line below drops restaurants that only appear once, we may no longer want this to happen
#Commented out for now, may make sense to use when considering outliers for averages
#tempDF = tempDF[sheets_result["Restaurant Name"].duplicated(keep=False) == True]
tempDF = tempDF.groupby(["Restaurant Name"]).agg({"Base": [np.mean, np.sum], 
                                                  "Tip": [np.mean, np.sum],
                                                  "Total": [np.mean, np.sum],
                                                 'Restaurant Name': 'size'})
tempDF.columns = ['Avg Base', 'Sum Base', 'Avg Tip', 
                  'Sum Tip', 'Avg Total', 'Sum Total', "Deliveries"]

tempDF = tempDF.round({'Avg Base': 2, 
                       'Sum Base': 2,
                       'Avg Tip': 2,
                       'Sum Tip': 2,
                       'Avg Total': 2,
                       'Sum Total': 2})

#Change sort value to see differences
tempDF.sort_values("Sum Total", ascending=False)



#RESTAURANTS DF

,Avg Base,Sum Base,Avg Tip,Sum Tip,Avg Total,Sum Total,Deliveries
Restaurant Name,,,,,,,
Chick-fil-A,5.70,689.75,3.61,436.39,9.31,1126.14,121
Taco Bell,4.79,315.87,3.44,226.75,8.70,574.12,66
McDonald's,4.59,316.50,3.30,227.42,7.90,544.92,69
Dave's Hot Chicken,5.24,256.75,4.36,213.50,9.60,470.25,49
Chipotle,4.69,248.50,3.52,186.42,8.24,436.92,53
...,...,...,...,...,...,...,...
Everbowl,4.00,4.00,2.00,2.00,6.00,6.00,1
Kung Fu Noodle,5.50,5.50,0.00,0.00,5.50,5.50,1
Papa Johns,4.00,4.00,1.00,1.00,5.00,5.00,1


In [19]:
# Overall hourly rate

hourlyRate = round(sheets_WeeksDF.iloc[-1]["Total"]/sheets_WeeksDF.iloc[-1]["Dash"], 2)
print("$", hourlyRate, "per hour, overall average")

$ 25.54 per hour, overall average


In [20]:
# Active time is time spent on deliveries, dash time is time spent on deliveries AND waiting for deliveries
# One factor that I wanted to consider in possible earnings maximization is:
# (Base + Tip) / Active time
# I've referred to this as the Pre Total : Active ratio
# Instead of something sensible like, active hourly pay rate

sheets_WeeksDF["pretotalactiveratio"] = round(sheets_WeeksDF["Pre Total"]/sheets_WeeksDF["Active"], 2)
sheets_WeeksDF.sort_values(by=["pretotalactiveratio"])

,Dates,Base,Tip,Adj pay,Other,Pre Total,Total,Active,Dash,Deliveries,Total Mileage,Active Mileage,Version,pretotalactiveratio
2,4/10 - 4/16,130.75,96.13,61.07,0.00,226.88,287.95,8.63,11.17,28,,,1,26.29
13,7/3 - 7/9,114.25,105.10,54.99,7.75,219.35,282.09,7.83,10.17,23,108,98.1,3,28.01
4,4/24 - 4/30,430.00,299.73,138.76,2.62,729.73,871.11,25.50,41.17,84,,,1,28.62
12,6/19 - 6/25,637.25,475.09,240.02,6.25,1112.34,1358.61,38.60,51.23,124,633,619.4,3,28.82
0,3/27 - 4/2,108.50,88.00,40.96,0.00,196.50,237.46,6.70,10.92,22,,,1,29.33
3,4/17 - 4/23,382.50,248.41,105.05,0.00,630.91,735.96,21.33,32.30,68,,,1,29.58
7,5/15 - 5/21,303.75,239.25,99.27,0.00,543.00,642.27,17.53,25.40,64,,,2,30.98
16,7/31 - 8/6,248.75,183.44,74.75,0.00,432.19,506.94,13.75,18.57,44,272,386.2,3,31.43
17,TOTALS:,5591.25,4399.11,1652.51,32.75,9990.36,11675.62,317.83,457.08,1068,2718,2615.88,-,31.43
10,6/5 - 6/11,197.25,185.09,65.38,0.00,382.34,447.72,11.77,15.75,41,183,161.6,3,32.48


In [21]:
#Dollars Earned vs Miles Driven is another factor I wanted to consider in the overall earnings equation

# Grab v3 deliveries, drop rows without known distance values
# Group by restaurant name, sum on total and distance
# Evaluate total pay over total distance as new column
# Sort low to high



tempDF = sheets_DelDF.copy()
tempDF = tempDF[tempDF.Distance!="-"]
tempDF["Distance"] = tempDF["Distance"].astype(float)
tempDF = tempDF.groupby(["Restaurant Name"]).agg({"Total": "sum", "Distance": "sum"})
tempDF = tempDF.round({'Base': 2, 'Tip': 2})
tempDF["DollarMilesRatio"] = tempDF["Total"]/tempDF["Distance"]
tempDF.sort_values(by="DollarMilesRatio")



#JOIN WITH RESTAURANTS DF

,Total,Distance,DollarMilesRatio
Restaurant Name,,,
PetSmart,44.55,148.8,0.299395
Walgreens,13.30,37.2,0.357527
Yukiya Sushi,8.25,10.8,0.763889
Pedro's Tacos,7.00,7.9,0.886076
El Bronco,9.25,9.7,0.953608
...,...,...,...
Ohana Hawaiian BBQ,8.25,1.8,4.583333
The Privateer Coal Fire Pizza,31.75,4.9,6.479592
Round Table Pizza,9.75,1.4,6.964286


In [35]:
#Need to drop non numerics to get max 'Total Mileage' - right now its 151 miles
#sheets_DaysDF[["Total", "Base", "Tip", "Active", "Dash", "Deliveries"]].max() 
#This isn't useful for later insights, but I find it interesting

v3DaysDF = sheets_DaysDF[sheets_DaysDF["Version"] == 3].copy()
sheets_DaysDF[["Total", "Base", "Tip", "Active", "Dash", "Deliveries"]].max() 

Total         333.57
Base          191.50
Tip           149.53
Active          9.67
Dash           13.60
Deliveries     33.00
dtype: float64

In [39]:
sheets_DaysDF[["Total", "Base", "Tip", "Active", "Dash", "Deliveries"]].mean()

Total         163.776393
Base           91.659836
Tip            72.116557
Active          5.198525
Dash            7.483115
Deliveries     17.508197
dtype: float64

In [40]:
v3DaysDF[["Total Mileage", "Active Mileage"]].max()
#v3DaysDF.dtypes

Total Mileage       151
Active Mileage    229.2
dtype: object

In [41]:
v3DaysDF[["Total Mileage", "Active Mileage"]].mean()
#v3DaysDF.dtypes

Total Mileage     97.071429
Active Mileage    93.425000
dtype: float64

In [27]:
sheets_DaysDF[["Total", "Base", "Tip", "Active", "Dash", "Deliveries"]].mean() 

Total         163.776393
Base           91.659836
Tip            72.116557
Active          5.198525
Dash            7.483115
Deliveries     17.508197
dtype: float64

In [30]:
#sheets_WeeksDF.head()
#Split into v3 (total mile, active mile) and all (base, tip, adj pay, pre total, total, active, dash, deliveries)

v3WeeksDF = sheets_WeeksDF1[sheets_WeeksDF1["Version"] == 3].copy()
AllWeeksDF = sheets_WeeksDF1.copy()

AllWeeksDF[["Base", "Tip", "Adj pay", "Pre Total", "Total", "Active", "Dash", "Deliveries"]].max()

Base           637.25
Tip            475.09
Adj pay        240.02
Pre Total     1112.34
Total         1358.61
Active          38.60
Dash            51.23
Deliveries     124.00
dtype: float64

In [31]:
AllWeeksDF[["Base", "Tip", "Adj pay", "Pre Total", "Total", "Active", "Dash", "Deliveries"]].mean()

Base          288.305556
Tip           242.637778
Adj pay        96.890000
Pre Total     530.943333
Total         630.486667
Active         16.718889
Dash           22.812222
Deliveries     55.000000
dtype: float64

In [32]:
v3WeeksDF[["Total Mileage", "Active Mileage"]].max()

Total Mileage       633
Active Mileage    619.4
dtype: object

In [33]:
v3WeeksDF[["Total Mileage", "Active Mileage"]].mean()

Total Mileage     302.000000
Active Mileage    290.653333
dtype: float64

In [23]:
# Also interested in longest order by duration and by distance
# Using sheets_DelDF1 because it has the nan value dropped
sheets_DelDF1[["Distance", "Duration"]].max()


Distance     37.20
Duration    128.00
Total        28.75
dtype: float64

In [24]:
sheets_DelDF1[["Distance", "Duration"]].mean()

Distance     5.284444
Duration    26.092929
Total        9.640384
dtype: float64

In [42]:
sheets_result[["Total"]].max()

Total    32.5
dtype: float64

In [43]:
sheets_result[["Total"]].mean()

Total    9.345519
dtype: float64

In [25]:
#Average tip, including and excluding untipped deliveries

TipAvg = round(sheets_result["Tip"].mean(), 2)
print("$", TipAvg, "average tip value including orders with no tip")
TipAvg2 = round(sheets_result[sheets_result["Tip"] !=0]["Tip"].mean(), 2)
print("$", TipAvg2, "average tip value excluding orders with no tip")

#This isn't useful for later insights, but I find it interesting

$ 4.12 average tip value including orders with no tip
$ 4.54 average tip value excluding orders with no tip


In [26]:
# Average $/hour and tip amount by day of week
# NOTE: This excludes pay adjustments

tempDF = sheets_DaysDF.copy()

# Can move this up to formatting function calls
# Also, check if necessary to convert from string to timestamp to datetime
date_to_timestamp(tempDF, "Date")
tempDF["Date"] = pd.to_datetime(tempDF["Date"])

#Adds column to DaysDF for the day of the week (Monday, Thursday, etc.)
tempDF["Day of Week"] = tempDF["Date"].dt.day_name()

tempDF = tempDF.groupby(["Day of Week"]).agg(
    {"Dash": "sum", 
     "Total": [np.mean, np.sum], 
     "Tip": "sum", 
     "Deliveries": "sum"})
tempDF.columns = ['Sum Dash', 'Avg Daily Total', 'Sum Total', 'Sum Tip', 'Sum Deliveries']

tempDF["$/hr"] = tempDF["Sum Total"]/tempDF["Sum Dash"]
tempDF["Avg Delivery Tip"] = tempDF["Sum Tip"]/tempDF["Sum Deliveries"]
tempDF["Avg Delivery Total"] = tempDF["Sum Total"]/tempDF["Sum Deliveries"]

tempDF = tempDF.round({'Sum Dash': 2, 
                       'Avg Total': 2, 
                       'Sum Total':2, 
                       'Sum Tip': 2,
                       '$/hr': 2,
                       'AvgTip': 2,
                       'AvgTotal':2})

display(tempDF)

,Sum Dash,Avg Daily Total,Sum Total,Sum Tip,Sum Deliveries,$/hr,Avg Delivery Tip,Avg Delivery Total
Day of Week,,,,,,,,
Friday,135.57,189.533125,3032.53,1385.78,316,22.37,4.385380,9.596614
Monday,17.85,125.823333,377.47,158.97,41,21.15,3.877317,9.206585
Saturday,112.83,185.460714,2596.45,1144.95,270,23.01,4.240556,9.616481
Sunday,49.58,147.347500,1178.78,545.53,122,23.78,4.471557,9.662131
Thursday,72.83,140.773636,1548.51,656.01,176,21.26,3.727330,8.798352
Wednesday,67.81,139.624444,1256.62,507.87,143,18.53,3.551538,8.787552
